## Interpolating the data

In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import RBFInterpolator, Rbf
from numpy.linalg import LinAlgError

In [2]:
# 1. Load data
df = pd.read_csv('../Data/CleanedData.csv')  # replace with your CSV file path
df.head()

,helical_angle,helical_layer_count,hoop_layer_count,bust_pressure,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers
0,14,3,3.0,519,1082.0,108.3,57.5,0.33,240.3,317,6,130,0.0
1,14,3,3.0,383,1082.0,108.3,57.5,0.33,240.3,317,6,130,0.0
2,14,3,6.0,438,1082.0,108.3,57.5,0.33,240.3,317,6,130,0.0
3,14,3,9.0,552,1082.0,108.3,57.5,0.33,240.3,317,6,130,0.0
4,14,6,6.0,700,1082.0,108.3,57.5,0.33,240.3,317,6,130,0.0


In [3]:
features = [
    'tensile_str', 'e1_gpa', 'youngs_modulus', 'poision_ratio',
    'yeild_strength', 'diameter', 'ult_tensile_strength',
    'helical_angle', 'helical_layer_count', 'hoop_layer_count',
    'liner_thickness', 'doily_layers'
]
target = 'bust_pressure'

In [4]:
X = df[features].values      # shape (49, 12)
y = df['bust_pressure'].values  # shape (49,)
X.shape, y.shape

((43, 12), (43,))

In [5]:
rbf_interp = None
smoothing = 0.0
for trial in range(6):  # try smoothing = 0.0, 1e-2, 1e-3, ...
    try:
        rbf_interp = RBFInterpolator(X, y, kernel='linear', smoothing=smoothing)
        print(f"✅ RBFInterpolator succeeded with smoothing={smoothing}")
        break
    except LinAlgError:
        smoothing = 10**(-trial-2)  # 1e-2, 1e-3, ..., 1e-7
else:
    print("⚠️ RBFInterpolator failed — using fallback Rbf model.")
    rbf_interp = None


✅ RBFInterpolator succeeded with smoothing=0.01


In [6]:
# 4. If still failed, fallback to Rbf (older but more tolerant)
use_old = False
if rbf_interp is None:
    rbf_old = Rbf(
        *[X[:, i] for i in range(X.shape[1])],
        y,
        function='multiquadric',
        epsilon=1.0,
        smooth=1e-2
    )
    use_old = True


In [7]:
new_samples = np.array([
    [400.0, 5.0,   200.0, 0.3, 350.0, 50.0, 500.0, 30.0, 10, 10, 2.0, 5],
    [450.0, 5.5,   210.0, 0.29, 360.0, 55.0, 520.0, 35.0, 12,  9, 1.8, 4]
])


In [8]:
if not use_old:
    y_pred = rbf_interp(new_samples)
else:
    y_pred = rbf_old(*[new_samples[:, i] for i in range(new_samples.shape[1])])

In [9]:
# 7. Display results in a DataFrame
result_df = pd.DataFrame(new_samples, columns=features)
result_df['predicted_burst_pressure'] = y_pred
print("\n🧪 Interpolation Results:\n")
print(result_df.to_string(index=False))


🧪 Interpolation Results:

 tensile_str  e1_gpa  youngs_modulus  poision_ratio  yeild_strength  diameter  ult_tensile_strength  helical_angle  helical_layer_count  hoop_layer_count  liner_thickness  doily_layers  predicted_burst_pressure
       400.0     5.0           200.0           0.30           350.0      50.0                 500.0           30.0                 10.0              10.0              2.0           5.0                995.266661
       450.0     5.5           210.0           0.29           360.0      55.0                 520.0           35.0                 12.0               9.0              1.8           4.0                995.611875


In [10]:
df.sample(10)  # Display the original DataFrame for reference

,helical_angle,helical_layer_count,hoop_layer_count,bust_pressure,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers
24,14,9,7.0,1235,1100.0,95.60,57.5,0.33,240.3,317,6,130,8.5
42,11,3,3.0,887,609.8,27.93,192.0,0.28,235.0,360,4,132,0.0
19,14,9,7.0,1018,1100.0,95.60,57.5,0.33,240.3,317,6,130,0.0
28,14,9,7.0,1036,1100.0,95.60,57.5,0.33,240.3,317,6,130,8.5
14,14,9,8.0,1050,1082.0,108.30,57.5,0.33,240.3,317,6,130,0.0
0,14,3,3.0,519,1082.0,108.30,57.5,0.33,240.3,317,6,130,0.0
17,14,15,11.0,880,1100.0,95.60,57.5,0.33,240.3,317,6,130,0.0
7,14,10,10.0,923,1082.0,108.30,57.5,0.33,240.3,317,6,130,0.0
25,14,9,7.0,1280,1100.0,95.60,57.5,0.33,240.3,317,6,130,8.5
9,14,9,9.0,735,1082.0,108.30,57.5,0.33,240.3,317,6,130,0.0


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
helical_angle,43.0,13.441860,1.181249,11.00,14.00,14.00,14.00,14.00
helical_layer_count,43.0,7.558140,3.238918,3.00,4.00,9.00,9.00,15.00
hoop_layer_count,43.0,6.651163,2.484559,3.00,6.00,7.00,8.00,11.00
bust_pressure,43.0,954.093023,297.302060,383.00,740.00,965.00,1158.00,1518.00
tensile_str,43.0,1002.520930,190.152511,609.80,1082.00,1082.00,1100.00,1100.00
e1_gpa,43.0,87.440465,29.354292,27.93,95.60,95.60,108.30,108.30
youngs_modulus,43.0,82.523256,52.959323,57.50,57.50,57.50,57.50,192.00
poision_ratio,43.0,0.320698,0.019687,0.28,0.33,0.33,0.33,0.33
yeild_strength,43.0,239.313953,2.086873,235.00,240.30,240.30,240.30,240.30
ult_tensile_strength,43.0,325.000000,16.931233,317.00,317.00,317.00,317.00,360.00


## Generating synthetic samples

In [12]:
HELICAL_ANGLE_LB = 10
HELICAL_ANGLE_UB = 50

HELICAL_LAYER_LB = 2
HELICAL_LAYER_UB = 17

HOOP_LAYER_LB = 1
HOOP_LAYER_UB = 15

TENSILE_STRENGTH_LB = 600
TENSILE_STRENGTH_UB = 1100

E1_LB = 20
E1_UB = 120

YOUNGS_MODULUS_LB = 55
YOUNGS_MODULUS_UB = 200

POISSONS_RATIO_LB = 0.25
POISSONS_RATIO_UB = 0.35

YEILD_STRENGTH_LB = 230
YEILD_STRENGTH_UB = 250

ULT_STRENGTH_LB = 310
ULT_STRENGTH_UB = 370

LINER_THICKNESS_LB = 0.2
LINER_THICKNESS_UB = 0.8

DIAMETER_LB = 100
DIAMETER_UB = 180

DOLIY_LB = 1
DOLIY_UB = 20

In [13]:
# from dependencies.constants import *

feature_ranges = {
    'helical_angle': (HELICAL_ANGLE_LB, HELICAL_ANGLE_UB),         # degrees
    'helical_layer_count': (HELICAL_LAYER_LB, HELICAL_LAYER_UB),        # integer
    'hoop_layer_count': (HOOP_LAYER_LB, HOOP_LAYER_UB),           # integer
    'tensile_str': (TENSILE_STRENGTH_LB, TENSILE_STRENGTH_UB),         # MPa
    'e1_gpa': (E1_LB, E1_UB),                  # GPa
    'youngs_modulus': (YOUNGS_MODULUS_LB, YOUNGS_MODULUS_UB),      # GPa
    'poision_ratio': (POISSONS_RATIO_LB, POISSONS_RATIO_UB),         # unitless
    'yeild_strength': (YEILD_STRENGTH_LB, YEILD_STRENGTH_UB),      # MPa
    'ult_tensile_strength': (ULT_STRENGTH_LB, ULT_STRENGTH_UB),# MPa
    'liner_thickness': (LINER_THICKNESS_LB, LINER_THICKNESS_UB),         # mm
    'diameter': (DIAMETER_LB, DIAMETER_UB),              # mm
    'doily_layers': (DOLIY_LB, DOLIY_UB)                 # integer
}

features = list(feature_ranges.keys())
features

['helical_angle',
 'helical_layer_count',
 'hoop_layer_count',
 'tensile_str',
 'e1_gpa',
 'youngs_modulus',
 'poision_ratio',
 'yeild_strength',
 'ult_tensile_strength',
 'liner_thickness',
 'diameter',
 'doily_layers']

In [14]:
# Generate N samples
N = 450  # You can change this
synthetic_samples = []

np.random.seed(123)  # For reproducibility

for _ in range(N):
    sample = []
    for feat in features:
        low, high = feature_ranges[feat]
        if isinstance(low, int) and isinstance(high, int):
            sample.append(np.random.randint(low, high+1))
        else:
            sample.append(np.random.uniform(low, high))
    synthetic_samples.append(sample)

X_synth = np.array(synthetic_samples)


In [15]:
if not use_old:
    y_synth = rbf_interp(X_synth)
else:
    y_synth = rbf_old(*[X_synth[:, i] for i in range(X_synth.shape[1])])

In [16]:
df_synth = pd.DataFrame(X_synth, columns=features)
df_synth['bust_pressure'] = y_synth
df_synth = df_synth.round(2)
df_synth.to_csv('../Data/InterpolatedData.csv', index=False)

df_synth.shape

(450, 13)

In [17]:
df_synth.head(10)

,helical_angle,helical_layer_count,hoop_layer_count,tensile_str,e1_gpa,youngs_modulus,poision_ratio,yeild_strength,ult_tensile_strength,liner_thickness,diameter,doily_layers,bust_pressure
0,12.0,14.0,3.0,830.0,37.0,138.0,0.30,231.0,342.0,0.55,147.0,10.0,992.41
1,42.0,16.0,1.0,711.0,45.0,138.0,0.27,234.0,342.0,0.56,149.0,4.0,989.93
2,12.0,6.0,8.0,922.0,104.0,102.0,0.31,246.0,317.0,0.63,152.0,7.0,992.15
3,43.0,7.0,7.0,950.0,47.0,89.0,0.28,238.0,313.0,0.26,164.0,12.0,994.02
4,44.0,12.0,11.0,878.0,97.0,70.0,0.29,242.0,340.0,0.51,126.0,17.0,995.50
5,16.0,16.0,5.0,959.0,95.0,184.0,0.30,235.0,365.0,0.51,156.0,18.0,997.08
6,22.0,4.0,2.0,601.0,71.0,172.0,0.28,247.0,332.0,0.61,111.0,8.0,991.01
7,44.0,5.0,12.0,859.0,114.0,61.0,0.27,244.0,348.0,0.73,170.0,13.0,996.10
8,37.0,8.0,2.0,1045.0,94.0,154.0,0.34,231.0,357.0,0.49,105.0,9.0,997.07
9,32.0,16.0,12.0,896.0,45.0,100.0,0.30,233.0,339.0,0.33,168.0,8.0,992.90
